<a href="https://colab.research.google.com/github/Data-Analytics-with-Python/individual-assignment-iii-ninivenus/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT: Before you start, enter your name and student number below.

**Full Name**: Chenxin Sun

**Student Number**: 400582270

# Predictive Analytics for Nata Supermarket

Welcome to Part III of our case assignment. In this part, we will continue working with the same dataset of **Nata Supermarket**.  Our focus here will be on performing predictive modeling tasks.

Throughout this assignment, please ensure that your results are reproducible by setting the **random_state to 20**



# Loading and preparing the data

To begin with, load the data as a `pandas` data frame. Recall that you there are missing values in the data. **Make sure to address** the following issues from part I of the assignment before starting your analysis:

* Remove the missing values
* Remove any column of constant values
* Convert the column `Dt_Customer` to number of days the customer has been with the company.

In [70]:
import pandas as pd
from datetime import datetime

df = pd.read_excel("Nata Supermarkets1.xlsx")

print("Initial shape:", df.shape)
df.head()


Initial shape: (2240, 29)


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-04-09 00:00:00,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-08-03 00:00:00,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-10-02 00:00:00,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [71]:
df = df.dropna()
print("After dropping missing values:", df.shape)

After dropping missing values: (2216, 29)


In [72]:
constant_cols = [col for col in df.columns if df[col].nunique() == 1]

df = df.drop(columns=constant_cols)
print("Removed constant columns:", constant_cols)
print("Shape after removing constant columns:", df.shape)


Removed constant columns: ['Z_CostContact', 'Z_Revenue']
Shape after removing constant columns: (2216, 27)


In [73]:
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"])

reference_date = df["Dt_Customer"].max()

df["Customer_Days"] = (reference_date - df["Dt_Customer"]).dt.days

df = df.drop(columns=["Dt_Customer"])

df.head()


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Customer_Days
0,5524,1957,Graduation,Single,58138.0,0,0,58,635,88,...,4,7,0,0,0,0,0,0,1,971
1,2174,1954,Graduation,Single,46344.0,1,1,38,11,1,...,2,5,0,0,0,0,0,0,0,125
2,4141,1965,Graduation,Together,71613.0,0,0,26,426,49,...,10,4,0,0,0,0,0,0,0,472
3,6182,1984,Graduation,Together,26646.0,1,0,26,11,4,...,4,6,0,0,0,0,0,0,0,65
4,5324,1981,PhD,Married,58293.0,1,0,94,173,43,...,6,5,0,0,0,0,0,0,0,321


In [74]:
RANDOM_STATE = 20
print("Random state set to:", RANDOM_STATE)


Random state set to: 20


# Question 1 (45 points)

In this question, we will predict a customer's spending amount on each product category over a two year period. Let us assume that when we try to predict a customer's spending on a product category (such as wines), their spending on other products is not observable.

In this question and Question 2, we will focus on **wines**.

(i). **Split the data** into two data frames, X (**features**) and y (**target**).

Then, further **split the data** into **training** and **testing** sets.

In [98]:
from sklearn.model_selection import train_test_split

y = df["MntWines"]

X = df.drop(columns=["MntWines"])

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RANDOM_STATE
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (1772, 27)
Test shape: (444, 27)


(ii). **Build a machine learning pipeline** that combines the following steps to predict spending amount on wines:

* Performing one-hot encoding for the categorical features;
* A random forest model for regression.

In the random forest model, **specify** the following hyperparameters:
* Number of trees;
* Maximum depth of any tree
* Minimum number of data points required to split a node;
* Minimum number of data points in any leaf node

In addition, **fit your model** to the training data.

In [76]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
    ],
    remainder="passthrough"
)

rf = RandomForestRegressor(
    n_estimators=200,        # Number of trees
    max_depth=10,           # Maximum depth of any tree
    min_samples_split=5,    # Minimum # of data points to split a node
    min_samples_leaf=2,     # Minimum # of data points in any leaf node
    random_state=RANDOM_STATE
)

rf_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", rf)
])

rf_pipeline.fit(X_train, y_train)

print("Random forest pipeline fitted on training data.")


Random forest pipeline fitted on training data.


(iii). Use the model to **predict** the spending amount on wines by a customer with the following features.

| Feature |  Value  |
|---------|---------|
| Age     | 48      |
|Education|Graduation|
|Marital_Status| Married|
|Income|80,000|
|Kidhome|1|
|Teenhome|1|
|Dt-Customer|2016-10-10|
|Recency|43|     
|NumDealsPurchases|2|
|   NumWebPurchases|1|
|NumCatalogPurchases|0|
|NumStorePurchases|15|
|NumWebVisitsMonth|5|
|AcceptedCmp1,2,3,4,5| 0 |
|Complain|0|




In [77]:
dt_customer = pd.to_datetime("2016-10-10")

customer_days_new = (reference_date - dt_customer).days


In [78]:
new_customer = pd.DataFrame({
    "ID": [0],
    "Year_Birth": [1966],
    "Education": ["Graduation"],
    "Marital_Status": ["Married"],
    "Income": [80000],
    "Kidhome": [1],
    "Teenhome": [1],
    "Recency": [43],
    "MntFruits": [0],
    "MntMeatProducts": [0],
    "MntFishProducts": [0],
    "MntSweetProducts": [0],
    "MntGoldProds": [0],
    "NumDealsPurchases": [2],
    "NumWebPurchases": [1],
    "NumCatalogPurchases": [0],
    "NumStorePurchases": [15],
    "NumWebVisitsMonth": [5],
    "AcceptedCmp1": [0],
    "AcceptedCmp2": [0],
    "AcceptedCmp3": [0],
    "AcceptedCmp4": [0],
    "AcceptedCmp5": [0],
    "Complain": [0],
    "Response": [0],
    "Customer_Days": [customer_days_new]
})


In [79]:
new_customer = new_customer[X.columns]


In [80]:
y_pred = rf_pipeline.predict(new_customer)
print("Predicted spending on wines for this customer:", y_pred[0])


Predicted spending on wines for this customer: 407.68180952380936


(iv). **Consider** **two** measures to evaluate the model's performance on the test dataset.

Based on you computational results, how would you describe the model's performance?

In [99]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Predict on test set
y_pred_test = rf_pipeline.predict(X_test)

# 1. Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred_test)

# 2. Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print("MAE:", mae)
print("RMSE:", rmse)


MAE: 89.55653832762953
RMSE: 147.19700708328344


To evaluate the model’s performance on the test dataset, two measures were used: Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE).

MAE = 89.56
This means that, on average, the model’s predictions are about 89 spending units away from the true wine spending values. MAE provides an intuitive measure of the typical prediction error.

RMSE = 147.20
RMSE is larger because it penalizes large errors more heavily. A value around 147 suggests that while the model performs reasonably well for most predictions, there are occasional cases where the prediction error is considerably larger.

Overall Model Performance:
The model shows moderate predictive accuracy. The error levels are not extremely high relative to the typical magnitude of wine spending, but the relatively large RMSE indicates that the model struggles with some customers who spend atypical amounts. In general, the random forest captures meaningful patterns in the data but still leaves room for improvement，possibly through additional feature engineering, hyperparameter tuning, or alternative modeling approaches.

(v). **Perform** a 6-fold cross validation with a performance score of your choice.

**Note**: You may need to research on how to specify the performance score for regression models.

In [81]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(
    rf_pipeline,
    X_train,
    y_train,
    cv=6,
    scoring="r2"
)

print("6-fold CV R^2 scores:", cv_scores)
print("Mean CV R^2:", cv_scores.mean())
print("Std CV R^2:", cv_scores.std())


6-fold CV R^2 scores: [0.80164613 0.78557415 0.82250496 0.78551797 0.79380269 0.79737021]
Mean CV R^2: 0.7977360195658529
Std CV R^2: 0.01252721110641262


(vi). **Perform** hyperparameter tuning using `GridSearchCV` for the following hyperparameters:

* Number of trees: 50, 100
* Maximum depth of any tree: 5, 10, 15
* Minimum number of data points required to split a node: 3, 6
* Minimum number of data points in any leaf node: 2,4,8


Based on your computational result, **show**:
* the best hyperparameter comination
* the corresponding performance score

In addition, **retrieve** the best model (the one corresponding to the best performance score).

In [82]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "model__n_estimators": [50, 100],
    "model__max_depth": [5, 10, 15],
    "model__min_samples_split": [3, 6],
    "model__min_samples_leaf": [2, 4, 8]
}

grid_search = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=param_grid,
    cv=6,
    scoring="r2",
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best hyperparameter combination:")
print(grid_search.best_params_)

print("\nBest cross-validated R^2 score:")
print(grid_search.best_score_)

best_model = grid_search.best_estimator_
print("\nBest model (pipeline):")
print(best_model)


Best hyperparameter combination:
{'model__max_depth': 15, 'model__min_samples_leaf': 2, 'model__min_samples_split': 3, 'model__n_estimators': 100}

Best cross-validated R^2 score:
0.8029174431638357

Best model (pipeline):
Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Education',
                                                   'Marital_Status'])])),
                ('model',
                 RandomForestRegressor(max_depth=15, min_samples_leaf=2,
                                       min_samples_split=3, random_state=20))])


# Question 2 (24 points)

In this question, we will compare the performance of the best model found through `GridSearchCV` in Question 1 with the performance of the linear regression model.

(i). **Construct** a linear regression model using all relevant features and fit it to the training data.

Further, **evaluate** the model's performance on the test data and compare it with the best random forest model found in Question 1, with respect to the two performance considered in Question 1.

**Note**: You may use the same training and testing datasets as in Question 1.

In [84]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1. Construct a linear regression model pipeline
linear_regression_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), # Reusing the preprocessor from RF pipeline
    ('regressor', LinearRegression())
])

# 2. Fit the linear regression model to the training data
linear_regression_pipeline.fit(X_train, y_train)
print("Linear Regression model fitted.")

# 3. Evaluate the Linear Regression model on the test data
y_pred_lin = linear_regression_pipeline.predict(X_test)

mae_lin = mean_absolute_error(y_test, y_pred_lin)
rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))
r2_lin = r2_score(y_test, y_pred_lin)

print("\n--- Linear Regression Model Performance ---")
print(f"MAE: {mae_lin:.2f}")
print(f"RMSE: {rmse_lin:.2f}")
print(f"R-squared: {r2_lin:.2f}")

# 4. Evaluate the best Random Forest model (from Q1) on the test data
y_pred_rf = best_model.predict(X_test)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print("\n--- Best Random Forest Model Performance (from Q1) ---")
print(f"MAE: {mae_rf:.2f}")
print(f"RMSE: {rmse_rf:.2f}")
print(f"R-squared: {r2_rf:.2f}")

print("\n--- Comparison ---")
if r2_rf > r2_lin:
    print(f"The Random Forest model performs better (R-squared: {r2_rf:.2f}) than the Linear Regression model (R-squared: {r2_lin:.2f}).")
else:
    print(f"The Linear Regression model performs better (R-squared: {r2_lin:.2f}) than the Random Forest model (R-squared: {r2_rf:.2f}).")


Linear Regression model fitted.

--- Linear Regression Model Performance ---
MAE: 119.71
RMSE: 180.67
R-squared: 0.67

--- Best Random Forest Model Performance (from Q1) ---
MAE: 87.51
RMSE: 144.90
R-squared: 0.79

--- Comparison ---
The Random Forest model performs better (R-squared: 0.79) than the Linear Regression model (R-squared: 0.67).


On the test data, the best Random Forest model performs better than the Linear Regression model.
It achieves lower MAE (about 87 vs. 120) and lower RMSE (about 145 vs. 181), and a higher R-squared (0.79 vs. 0.67).
This means the Random Forest explains more variance in wine spending and provides more accurate predictions than the Linear Regression model.

(ii). Let's further compare the distribution of prediction errors by the two models in the following steps.

**Step 1**. For both the linear regression and the (best) random forest model, compute the absolute residual residual for each record in the test dataset.
  * Note that the absolute residual is distance between the predicted value and actual value, i.e., $|y_{pred}-y_{test}|$.

So we end up with two sets of absolute residuals (one by the linear regression model and the other by the random forest model).

**Step 2**. For each pair of absolute residuals for the same test data point, we can define a point in a scatterplot. Genereate such a scatterplot using `plotly.express` (LR residuals vs. RF residuals).  

**Step 3**. Add a 45 degree reference line to the plot. This can be done using the following codes. (You may need to change `min_val` and `max_val` for better visualization).

```
min_val = 0
max_val = 10
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)
```

**Implement the above steps**.

Note that the above steps essentially creates a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). How would you interpret the plot (for comparing the two predictive models).

In [87]:
abs_res_lin = np.abs(y_pred_lin - y_test)
abs_res_rf = np.abs(y_pred_rf - y_test)


In [88]:
import plotly.express as px

fig = px.scatter(
    x = abs_res_lin,
    y = abs_res_rf,
    labels={'x': 'LR Absolute Residual', 'y': 'RF Absolute Residual'},
    title='Comparison of Absolute Residuals (LR vs RF)'
)


In [89]:
min_val = 0
max_val = max(abs_res_lin.max(), abs_res_rf.max())

fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash")
)

fig.show()


The scatterplot compares the absolute residuals produced by the linear regression model and the best random forest model for each observation in the test dataset. Each point represents a single test case, with the x-axis showing the linear regression’s absolute error and the y-axis showing the random forest’s absolute error.

The 45-degree dashed red line serves as a reference:

Points above the line indicate cases where the random forest made a larger error than linear regression.

Points below the line indicate cases where the random forest made a smaller error.

From the plot, most points lie below the diagonal line. This pattern shows that, for the majority of test observations, the random forest model produced smaller absolute residuals compared with the linear regression model. Even in regions with larger errors, the random forest typically remains closer to the diagonal or below it, suggesting more consistent performance across different types of customers.

Overall, the plot supports the earlier numerical comparison:
the random forest model generally yields lower prediction errors and offers better predictive accuracy than the linear regression model.

# Question 3. (24 points)

In this question, we will consider a classification problem on customers' spendings on meat products.


(i). For the column that represents customers' spendings on meat products, **calculate** the 33.33% and 66.67% percentiles. (**Hint**: You may use the function `df['MntMeatProducts'].quantile([1/3,2/3])`.)

Based on the two percentiles, **label** each row in the dataset
* If a customer's spending is below the 33.33% percentile, label their spending as "low";

* If a customer's spending is above the 66.67% percentile, label their spending as "high";


* If a customer's spending is between the two percentiles, label their spending as "medium".

In [90]:
q_low, q_high = df['MntMeatProducts'].quantile([1/3, 2/3])

print("33.33% percentile:", q_low)
print("66.67% percentile:", q_high)

def label_meat_spending(x):
    if x < q_low:
        return "low"
    elif x > q_high:
        return "high"
    else:
        return "medium"

df['Meat_Spend_Level'] = df['MntMeatProducts'].apply(label_meat_spending)

df[['MntMeatProducts', 'Meat_Spend_Level']].head()


33.33% percentile: 24.0
66.67% percentile: 146.33333333333258


,MntMeatProducts,Meat_Spend_Level
0,546,high
1,6,low
2,127,medium
3,20,low
4,118,medium


(ii). **Build a K-nearest-neighbors model** to predict the label of a customer's spending on meat products. The model should be part of a machine learning pipeline that preprocesses the data.





In [92]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

X = df.drop(columns=['Meat_Spend_Level'])
y = df['Meat_Spend_Level']

categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])

knn_pipeline.fit(X_train, y_train)

y_pred = knn_pipeline.predict(X_test)

print("KNN pipeline fitted and predictions made.")


KNN pipeline fitted and predictions made.


(iii). Evaluate the performace of your model in part (ii) on the test data by **generating the classification report**.

Further, **interpret** each number in the classification report based on the current context.

In [96]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)

print("Classification Report:\n", report)


Classification Report:
               precision    recall  f1-score   support

        high       0.89      0.83      0.86       222
         low       0.79      0.91      0.85       219
      medium       0.73      0.66      0.69       224

    accuracy                           0.80       665
   macro avg       0.80      0.80      0.80       665
weighted avg       0.80      0.80      0.80       665



The model achieves an overall accuracy of 80%, which suggests that the KNN classifier performs reasonably well in predicting customers’ meat-spending levels. Looking at the three classes individually:

High spenders
Precision = 0.89 means that when the model predicts “high”, it is correct 89% of the time.
Recall = 0.83 indicates that it successfully identifies 83% of true high-spending customers.
This class has the strongest performance.

Low spenders
Precision = 0.79 means that around 79% of predicted “low” labels are correct.
Recall = 0.91 shows that the model captures most actual low-spending customers.
This class benefits from higher recall than precision.

Medium spenders
Precision = 0.73 and recall = 0.66 indicate weaker performance.
The confusion likely comes from the medium group being more similar to high and low categories.

The macro and weighted averages are both 0.80, showing stable performance across classes despite class-size differences.

Overall, the model performs best on the high and low categories, while the medium category is more challenging for the classifier.

# Describe how you used Gen. AI. in this assignment (2 points)

I used generative AI as a supportive tool to clarify instructions and confirm my understanding of the assignment tasks. It helped me check the structure of my code and ensure that my interpretation of each question was correct. All coding decisions and implementation were completed by myself.

**Note**: The remaining 5 points will be assigned to readability of the work.